In [1]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_community.chat_models import ChatOllama
from services import create_chroma_client

In [3]:
oembed = OllamaEmbeddings(model="mistral")
db = Chroma(client=create_chroma_client(), embedding_function=oembed, collection_name="news")
retriever = db.as_retriever()

In [10]:
def retriever(query: dict):
    import time
    import datetime as dt

    fecha_dt = dt.datetime.today() - dt.timedelta(days=1)

    fecha_unix = time.mktime(fecha_dt.timetuple())
    documents = db.similarity_search(query['input'], k=20, filter={'published': {'$gt': fecha_unix}})

    if len(documents) > 0:
        return [document.page_content for document in documents]
    return []

In [11]:
template = """A partir de los siguientes titulares distintos portales de noticias, elabora una respuesta a la pregunta del usuario:

{context}

Pregunta: {question}

Recuerda que debes responder en español.
"""
rag_prompt = ChatPromptTemplate.from_template(template)

model = ChatOllama(
    model="openchat",
    temperature=0,
    streaming=True,
)
chain = (
    RunnableParallel({"context": retriever, "question": RunnablePassthrough()})
    | rag_prompt
    | model
    | StrOutputParser()
)

In [12]:
print(chain.invoke({'input': "Que paso ayer en gran hermano?"}))

 Ayer en Gran Hermano, hubo varios eventos destacados. Entre ellos, se mencionan el ingreso de un celular a la casa y los jugadores recibieron una dura advertencia. También hubo enfrentamientos entre los participantes, como cuando Catalina Gorostidi destrozó a Julieta Poggio. Además, se produjo un desesperado pedido de Furia a la producción de Gran Hermano y la hermana de Furia Scaglione denunció públicamente a LAM por acosos.
